<a href="https://colab.research.google.com/github/carokhan/apriltags/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive',force_remount=True)

import os 

os.chdir("/content/drive/My Drive")

Mounted at /content/drive


In [79]:
import pandas as pd
from tensorflow.keras.applications import MobileNet
import numpy as np
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
import math
import cv2
from keras.utils import Sequence
import tensorflow as tf
from tqdm.keras import TqdmCallback
import os
import datetime
from tensorflow.keras.callbacks import Callback

outs = ['center_score', 'center_link_score',
       'ordered_corner_0_x', 'ordered_corner_0_y', 'ordered_corner_1_x',
       'ordered_corner_1_y', 'ordered_corner_2_x', 'ordered_corner_2_y',
       'ordered_corner_3_x', 'ordered_corner_3_y', 'corner_anchor_0_x',
       'corner_anchor_0_y', 'corner_anchor_1_x', 'corner_anchor_1_y',
       'corner_anchor_2_x', 'corner_anchor_2_y', 'corner_anchor_3_x',
       'corner_anchor_3_y', 'corner_score_0', 'corner_score_1',
       'corner_score_2', 'corner_score_3', 'center_pos_x', 'center_pos_y']

class DataGenerator(Sequence):
    def __init__(self, df, batch_size=16):
        self.df = df
        self.batch_size = batch_size
        self.labels = [[row[y] for y in outs] for (idx, row) in df.iterrows()]
        self.im_list = self.df['image'].tolist()

    def __len__(self):
        return int(math.ceil(len(self.df) / float(self.batch_size)))

    def get_batch_labels(self, idx):
        return self.labels[idx * self.batch_size: (idx + 1) * self.batch_size]

    def get_batch_features(self, idx):
        return [cv2.imread(im) for im in self.im_list[idx * self.batch_size: (1 + idx) * self.batch_size]]

    def __getitem__(self, idx):
        batch_x = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)
        return np.array(batch_x), np.array(batch_y)

class MergeMetrics(Callback):

    def __init__(self,**kargs):
        super(MergeMetrics,self).__init__(**kargs)

    def on_train_batch_end(self, epoch, logs={}):
        logs["merged_acc"] = logs["center_link_score_accuracy"] *0.5 + logs["corner_score_0_accuracy"] *0.125 + logs["corner_score_1_accuracy"]*0.125 +  logs["corner_score_2_accuracy"]*0.125 +  logs["corner_score_3_accuracy"]*0.125
        logs["merged_mse"] = logs["ordered_corner_1_x_mse"] + logs["ordered_corner_1_y_mse"] + logs["ordered_corner_2_x_mse"] + logs["ordered_corner_2_y_mse"] + logs["ordered_corner_3_x_mse"] + logs["ordered_corner_3_y_mse"] + logs["corner_anchor_0_x_mse"] + logs["corner_anchor_0_y_mse"] + logs["corner_anchor_1_x_mse"] + logs["corner_anchor_1_y_mse"] + logs["corner_anchor_2_x_mse"] + logs["corner_anchor_2_y_mse"] + logs["corner_anchor_3_x_mse"] + logs["corner_anchor_3_y_mse"] + logs["center_pos_x_mse"] + logs["center_pos_y_mse"]

    def on_test_batch_end(self, epoch, logs={}):
        logs["merged_acc"] = logs["center_link_score_accuracy"] *0.5 + logs["corner_score_0_accuracy"] *0.125 + logs["corner_score_1_accuracy"]*0.125 +  logs["corner_score_2_accuracy"]*0.125 +  logs["corner_score_3_accuracy"]*0.125
        logs["merged_mse"] = logs["ordered_corner_1_x_mse"] + logs["ordered_corner_1_y_mse"] + logs["ordered_corner_2_x_mse"] + logs["ordered_corner_2_y_mse"] + logs["ordered_corner_3_x_mse"] + logs["ordered_corner_3_y_mse"] + logs["corner_anchor_0_x_mse"] + logs["corner_anchor_0_y_mse"] + logs["corner_anchor_1_x_mse"] + logs["corner_anchor_1_y_mse"] + logs["corner_anchor_2_x_mse"] + logs["corner_anchor_2_y_mse"] + logs["corner_anchor_3_x_mse"] + logs["corner_anchor_3_y_mse"] + logs["center_pos_x_mse"] + logs["center_pos_y_mse"]

    def on_epoch_begin(self,epoch, logs={}):
        return

    def on_epoch_end(self, epoch, logs={}):
        logs["merged_acc"] = logs["center_link_score_accuracy"] *0.5 + logs["corner_score_0_accuracy"] *0.125 + logs["corner_score_1_accuracy"]*0.125 +  logs["corner_score_2_accuracy"]*0.125 +  logs["corner_score_3_accuracy"]*0.125
        logs["merged_mse"] = logs["ordered_corner_1_x_mse"] + logs["ordered_corner_1_y_mse"] + logs["ordered_corner_2_x_mse"] + logs["ordered_corner_2_y_mse"] + logs["ordered_corner_3_x_mse"] + logs["ordered_corner_3_y_mse"] + logs["corner_anchor_0_x_mse"] + logs["corner_anchor_0_y_mse"] + logs["corner_anchor_1_x_mse"] + logs["corner_anchor_1_y_mse"] + logs["corner_anchor_2_x_mse"] + logs["corner_anchor_2_y_mse"] + logs["corner_anchor_3_x_mse"] + logs["corner_anchor_3_y_mse"] + logs["center_pos_x_mse"] + logs["center_pos_y_mse"]

tf.debugging.set_log_device_placement(False)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
# from tensorflow.python.profiler import profiler_client

# tpu_profile_service_address = os.environ['COLAB_TPU_ADDR'].replace('8470', '8466')
# print(profiler_client.monitor(tpu_profile_service_address, 100, 2))
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.TPUStrategy(tpu)

Tensorflow version 2.9.2
Running on TPU  ['10.42.150.226:8470']
  Timestamp: 05:01:18
  TPU type: TPU v2
  Utilization of TPU Matrix Units (higher is better): 0.000%




In [68]:
!wget https://85df-72-82-59-111.ngrok.io/final_data.zip -O ../../final_data.zip
!unzip /content/final_data.zip -d /content/final_zip

path = "/content/final_zip/final_data/"
df = pd.read_csv(os.path.join(path, "colab_data.csv"))
shuffled = df.sample(frac=1).reset_index(drop=True)

training_set = shuffled[:8000]
validation_set = shuffled[8000:10000]
test_set = shuffled[10000:]

In [69]:
mnet = MobileNet()
mnet.trainable = False
mnet_out = mnet.layers[-4].output

In [70]:
outputs = []
losses = []
loss_wts = []
metrics = {}
for name in outs:
    activaton = 'softmax' #'softmax' if 'score' in name else 'linear'
    loss = 'mse' #'binary_crossentropy' if 'score' in name else 'mse'
    metric =  'accuracy' if 'score' in name else 'mse'
    metrics[name] = metric
    losses.append(loss)
    if name != "center_link_score":
        loss_wts.append(0.5)
    else:
        loss_wts.append(1)
    t = Dense(49, activation='relu', name='pre_'+name)(mnet_out)
    outputs.append(Dense(1, activation=activaton, name=name)(t))
model = Model(inputs=mnet.input, outputs=outputs)
# model.compile(loss=losses, optimizer="adam")
model.compile(loss=losses, optimizer="adam", metrics=metrics, loss_weights=loss_wts)

In [80]:
checkpoint = MergeMetrics()

callbacks = [
            tf.keras.callbacks.ModelCheckpoint("apriltag_model", monitor="center_link_score_accuracy", mode="max", save_best_only=True),
            checkpoint
            ]
train_gen = DataGenerator(training_set, batch_size=64)
validate_gen = DataGenerator(validation_set, batch_size=64)

In [81]:
history = model.fit(x=train_gen, epochs=3, verbose=1, validation_data=validate_gen, steps_per_epoch=125, use_multiprocessing=False, callbacks=callbacks)


Epoch 1/3


NotFoundError: ignored

--2022-10-19 03:03:44--  https://85df-72-82-59-111.ngrok.io/final_data.zip
Resolving 85df-72-82-59-111.ngrok.io (85df-72-82-59-111.ngrok.io)... 3.134.125.175, 2600:1f16:d83:1201::6e:1
Connecting to 85df-72-82-59-111.ngrok.io (85df-72-82-59-111.ngrok.io)|3.134.125.175|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20040157140 (19G) [application/zip]
Saving to: ‘../../final_data.zip’

../../final_data.zi 100%[===================>]  18.66G  5.90MB/s    in 50m 35s 

2022-10-19 03:54:20 (6.30 MB/s) - ‘../../final_data.zip’ saved [20040157140/20040157140]

